In [1]:
import torch
from torch_geometric.datasets import MovieLens1M

# Load MovieLens 1M dataset (adjust root if needed)
dataset = MovieLens1M(root='/usr/itetnas04/data-scratch-01/trachsele/data/tl4rec/temp_pyg')
print(len(dataset))


Processing...


1


Done!


In [2]:
graph = dataset[0]
print(graph['user', 'rates', 'movie'].edge_index)


tensor([[   0,    0,    0,  ..., 6039, 6039, 6039],
        [1176,  655,  902,  ...,  558, 1080, 1081]])


In [3]:
print(graph)

HeteroData(
  movie={ x=[3883, 18] },
  user={ x=[6040, 30] },
  (user, rates, movie)={
    edge_index=[2, 1000209],
    rating=[1000209],
    time=[1000209],
  },
  (movie, rated_by, user)={
    edge_index=[2, 1000209],
    rating=[1000209],
    time=[1000209],
  }
)


In [20]:
#reading the edges
edges = graph['user', 'rates', 'movie'].edge_index

triplets = []
for i in range(edges.size(1)):
    u, v = edges[:, i]  # Get the user and movie node index
    triplets.append((u.item(),0, v.item()))

print(triplets[:10])


[(0, 0, 1176), (0, 0, 655), (0, 0, 902), (0, 0, 3339), (0, 0, 2286), (0, 0, 1179), (0, 0, 1267), (0, 0, 2735), (0, 0, 590), (0, 0, 907)]


In [22]:
import random

random.shuffle(triplets)
num_train = int(0.8 * len(triplets))
num_val = int(0.1 * len(triplets))

train_triplets = triplets[:num_train]
val_triplets = triplets[num_train:num_train + num_val]
test_triplets = triplets[num_train + num_val:]


# Save them into separate files
def save_triplets(filename, triplets):
    with open(filename, "w") as f:
        for u, v, e in triplets:
            f.write(f"{u}\t{v}\t{e}\n")

save_triplets("train.txt", train_triplets)
save_triplets("valid.txt", val_triplets)
save_triplets("test.txt", test_triplets)
print ("done")


done


In [ ]:
#MovieLens1M_pyG CoDExSmall

In [1]:
%run script/run.py -c config/transductive/inference.yaml --dataset CoDExSmall --epochs 0 --bpe null --gpus [0] --ckpt /usr/itetnas04/data-scratch-01/trachsele/data/tl4rec/ckpts/ultra_4g.pth

14:39:11   Random seed: 1024
14:39:11   Config file: config/transductive/inference.yaml
14:39:11   {'checkpoint': '/usr/itetnas04/data-scratch-01/trachsele/data/tl4rec/ckpts/ultra_4g.pth',
 'dataset': {'class': 'CoDExSmall', 'root': '~/git/ULTRA/kg-datasets/'},
 'model': {'class': 'Ultra',
           'entity_model': {'aggregate_func': 'sum',
                            'class': 'EntityNBFNet',
                            'hidden_dims': [64, 64, 64, 64, 64, 64],
                            'input_dim': 64,
                            'layer_norm': True,
                            'message_func': 'distmult',
                            'short_cut': True},
           'relation_model': {'aggregate_func': 'sum',
                              'class': 'RelNBFNet',
                              'hidden_dims': [64, 64, 64, 64, 64, 64],
                              'input_dim': 64,
                              'layer_norm': True,
                              'message_func': 'distmult',
    

Load rspmm extension. This may take a while...


14:39:28   mr: 38.5285
14:39:29   mrr: 0.477777
14:39:29   hits@1: 0.37329
14:39:29   hits@3: 0.529557
14:39:29   hits@10: 0.676793
14:39:29   >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
14:39:29   Evaluate on test
14:39:38   mr: 42.3884
14:39:38   mrr: 0.463778
14:39:38   hits@1: 0.360503
14:39:38   hits@3: 0.514497
14:39:38   hits@10: 0.665208


In [1]:
%run script/run.py -c config/transductive/inference.yaml --dataset MovieLens1M_pyG --epochs 0 --bpe null --gpus [0] --ckpt /usr/itetnas04/data-scratch-01/trachsele/data/tl4rec/ckpts/ultra_4g.pth

14:49:49   Random seed: 1024
14:49:49   Config file: config/transductive/inference.yaml
14:49:49   {'checkpoint': '/usr/itetnas04/data-scratch-01/trachsele/data/tl4rec/ckpts/ultra_4g.pth',
 'dataset': {'class': 'MovieLens1M_pyG', 'root': '~/git/ULTRA/kg-datasets/'},
 'model': {'class': 'Ultra',
           'entity_model': {'aggregate_func': 'sum',
                            'class': 'EntityNBFNet',
                            'hidden_dims': [64, 64, 64, 64, 64, 64],
                            'input_dim': 64,
                            'layer_norm': True,
                            'message_func': 'distmult',
                            'short_cut': True},
           'relation_model': {'aggregate_func': 'sum',
                              'class': 'RelNBFNet',
                              'hidden_dims': [64, 64, 64, 64, 64, 64],
                              'input_dim': 64,
                              'layer_norm': True,
                              'message_func': 'distmult',

Load rspmm extension. This may take a while...


14:50:32   mr: 792.05
14:50:32   mrr: 0.0139796
14:50:32   hits@1: 0
14:50:32   hits@3: 0.005
14:50:32   hits@10: 0.035
14:50:32   >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
14:50:32   Evaluate on test
14:50:39   mr: 772.72
14:50:39   mrr: 0.00928851
14:50:39   hits@1: 0
14:50:39   hits@3: 0.005
14:50:39   hits@10: 0.02


In [1]:
import numpy as np
import torch

print(np.__version__)  # Check NumPy version
print(torch.__version__)  # Check PyTorch version
print(torch.cuda.is_available())  # Check if CUDA is available


1.24.3
2.1.0
True


In [3]:
print(os.getcwd())
os.chdir("/usr/itetnas04/data-scratch-01/trachsele/data/tl4rec") 
print(os.getcwd())

/home/trachsele/git/ULTRA/output/Ultra/CoDExSmall/2024-10-12-14-39-11
/usr/itetnas04/data-scratch-01/trachsele/data/tl4rec


In [13]:
print(os.listdir())

['valid.txt', 'requirements.txt', 'MovieLense_load.ipynb', 'temp_pyg', 'config', '.git', '.gitignore', 'ultra', 'LICENSE', 'script', 'test.txt', 'train.txt', 'ckpts', 'asset', 'README.md', '.ipynb_checkpoints']


In [4]:
import requests

# URLs of the triplet files
urls = {
    "train": "https://raw.githubusercontent.com/Elematre/tl4rec/refs/heads/main/train.txt",
    "valid": "https://raw.githubusercontent.com/Elematre/tl4rec/refs/heads/main/valid.txt",
    "test": "https://raw.githubusercontent.com/Elematre/tl4rec/refs/heads/main/test.txt",
}

# Function to load triplet data from a URL
def load_triplets(url):
    response = requests.get(url)
    response.raise_for_status()  # Ensure we got the data correctly
    triplets = [line.strip().split() for line in response.text.splitlines()]
    return triplets

# Function to extract the set of unique nodes from triplets
def get_unique_nodes(triplets):
    nodes = set()
    for triplet in triplets:
        u, e, v = triplet
        nodes.add(u)
        nodes.add(v)
    return nodes

# Load triplet data from URLs
train_triplets = load_triplets(urls["train"])
valid_triplets = load_triplets(urls["valid"])
test_triplets = load_triplets(urls["test"])

# Extract unique nodes
train_nodes = get_unique_nodes(train_triplets)
valid_nodes = get_unique_nodes(valid_triplets)
test_nodes = get_unique_nodes(test_triplets)

# Find missing nodes in valid and test datasets
missing_in_valid = valid_nodes - train_nodes
missing_in_test = test_nodes - train_nodes

# Print results
if missing_in_valid:
    print(f"Nodes in valid but not in train: {missing_in_valid}")
else:
    print("All nodes in valid are in train.")

if missing_in_test:
    print(f"Nodes in test but not in train: {missing_in_test}")
else:
    print("All nodes in test are in train.")


All nodes in valid are in train.
All nodes in test are in train.
